# TiXI Tutorial 1: Read and write CPACS files

In this tutorial we will learn how to use the [*tixi*](https://github.com/DLR-SC/tixi) library to:
- initialize the *tixi* library
- read data from a CPACS file
- modify the data
- write the data back to a new CPACS file

Here are some helpful resources for [TiXI](https://github.com/DLR-SC/tixi):
- Binary Downloads:  https://github.com/DLR-SC/tixi/wiki/Downloads
- API Documentation: https://dlr-sc.github.io/tixi/
- Issue Tracker:     https://github.com/DLR-SC/tixi/issues
- Wiki:              https://github.com/DLR-SC/tixi/wiki

## Initialize TiXI
The first thing we need to do is to import and initialize tixi:

In [ ]:
from tixi3 import tixi3wrapper
tixi_h = tixi3wrapper.Tixi3()

Let's take a look into this object and see what functionality it provides using the [python help method](https://docs.python.org/3/library/functions.html#help):

In [ ]:
help(tixi_h)

## Open a CPACS File

We can now open a CPACS file from our `resources` with *tixi*:

In [ ]:
tixi_h.open('resources/CPACS_simple.xml')

You can also go to that directory and open the file to explore its contents.
It contains some basic description of an aircraft model, data on different airports, and a flightpath. In this tutorial we will use the `airports` and `flightPath` elements as an example of how to read, modify, and write data.

Now that the model is loaded into memory, we can start working with it.

## Read Data from the CPACS Model

First we are going to read some data from the model. Reading is always done using one of the `tixi_h.get...()` methods which take an [XPath](https://en.wikipedia.org/wiki/XPath) as their first argument. This `XPath` is basically a query that identifies `Elements` or `Attributes` within a XML file.

Let us get the `name` of the Frankfurt airport using the path to the airports and selecting it among all the other airports via its index:

In [ ]:
airport_name = tixi_h.getTextElement('/cpacs/airports/airport[1]/name')
print('Name: ', airport_name)

A selection via the index could be error prone and we can use the `uID` attribute instead:

In [ ]:
airport_name = tixi_h.getTextElement('/cpacs/airports/airport[@uID="FRA"]/name')
print('Name: ', airport_name)

If you already know the `uID` attribute of the element you are searching for, you can also use a special method in *tixi* to find the `XPath` for that element:

In [ ]:
airport_xpath = tixi_h.uIDGetXPath('FRA')
print('Airport XPath: ', airport_xpath)

We can then use this `XPath` to get the same information as above:

In [ ]:
airport_name = tixi_h.getTextElement('/'.join((airport_xpath, 'name')))    # join is equivalent to: airport_xpath + '/' + 'name'
print('Name: ', airport_name)

---

Let us now get some information about that airport from that CPACS model:

In [ ]:
positionNorth = tixi_h.getDoubleElement('/'.join((airport_xpath, 'positionNorth')))
positionEast = tixi_h.getDoubleElement('/'.join((airport_xpath, 'positionEast')))

print('Position North: %.3f' % positionNorth)
print('Position East: %.3f' % positionEast)

Here we are using the method `getDoubleElement` that automatically converts the data at the given `XPath` into a number.

### Reading Vector Data

Some information would be very long if it would be stored as XML elements, such as waypoint data on a specific flight path. This is why the special [Vector Type](https://www.cpacs.de/documentation/CPACS_3_4_0_Docs/html/32aea7db-266a-6dfa-5d16-f9b63c1e62a8.htm) exists in CPACS and *tixi* allows us to easily extract that type of data:

In [ ]:
# get the element we are interested in, in this case a flight-path
fp_xpath = '/'.join((tixi_h.uIDGetXPath('FRAtoEWRID'),'flightPath'))

# figure out the size of the data, i.e. the number of waypoints
vector_size = tixi_h.getVectorSize('/'.join((fp_xpath,'waypoints')))

# get the data we are interested in
waypoints = tixi_h.getTextElement('/'.join((fp_xpath,'waypoints'))).split(';')
latitude = tixi_h.getFloatVector('/'.join((fp_xpath,'latitude')), vector_size)
longitude = tixi_h.getFloatVector('/'.join((fp_xpath,'longitude')), vector_size)

# print the data
for i, wp in enumerate(waypoints):
    print(f'{wp:>10} {latitude[i]:>10.3f} {longitude[i]:>10.3f}')

## Write Data to a CPACS Model

After we have seen how to read data from a CPACS model, let us now write data to it!

Let's add information about Braunschweig airport:
- IATA code is `BWE` which we will use as the `uID`
- the `name` will be `Braunschweig, Germany`
- its position is `52.3199°N 10.556°E`
- the elevation is at `84 m`

In [ ]:
# identify the element below which we want to create the new airport
airports_xpath = '/cpacs/airports'

# count the current number of child-elements to know the next index
next_index = tixi_h.getNamedChildrenCount(airports_xpath,'airport') + 1

# create a new element at that index
tixi_h.createElementAtIndex(airports_xpath,'airport',next_index)

# get the xpath of that new element
braunschweig_xpath = '/'.join((airports_xpath,f'airport[{next_index}]'))

# add new attributes and elements
tixi_h.addTextAttribute(braunschweig_xpath,'uID','BWE')
tixi_h.addTextElement(braunschweig_xpath, 'name', 'Braunschweig, Germany')
tixi_h.addDoubleElement(braunschweig_xpath, 'positionNorth', 52.319, '%.3f')
tixi_h.addDoubleElement(braunschweig_xpath, 'positionEast', 10.556, '%.3f')
tixi_h.addIntegerElement(braunschweig_xpath, 'elevation', 84, '%i')

## Save a CPACS File

After we have modified the CPACS model using *tixi*, we can save it as a new CPACS file:

In [ ]:
tixi_h.save('tixi_tutorial_1_result.xml')
tixi_h.close()

Don't forget to call the `tixi_h.close()` method after saving!

## Next Steps

You have seen how seen how to open, modify, and save a CPACS file using *tixi*. Now you can try it out yourself with our [first exercise](../exercises/tixi_exercise_1.ipynb).